In [ ]:
from IPython.display import display, clear_output
import utils
import widgets
import ipywidgets as ipw

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
DATA_MODEL = utils.read_yaml(
    "/home/jovyan/aiida-openbis/Notebooks/Metadata_Schemas_LinkML/materialMLinfo.yaml"
)
# CONFIG_ELN = utils.read_json("eln_config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(
    CONFIG_ELN["url"], CONFIG_ELN["token"]
)

slabs_options = [
    DATA_MODEL["classes"][object_type]["title"] for object_type in CONFIG["slabs"]
]
slabs_options.insert(0, "No material")
material_selection_radio_button = utils.Radiobuttons(
    description="",
    options=slabs_options,
    disabled=False,
    layout=ipw.Layout(width="300px"),
    style={"description_width": "100px"},
)

material_selector = widgets.MaterialSelectionWidget()

sample_out_name_textbox = utils.Text(
    description="Name",
    disabled=False,
    layout=ipw.Layout(width="400px"),
    placeholder="Write sample name here...",
    style={"description_width": "110px"},
)
increase_buttons_size = utils.HTML(data="".join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description="",
    disabled=False,
    button_style="",
    tooltip="Save",
    icon="save",
    layout=ipw.Layout(width="100px", height="50px"),
)
quit_button = utils.Button(
    description="",
    disabled=False,
    button_style="",
    tooltip="Main menu",
    icon="home",
    layout=ipw.Layout(width="100px", height="50px"),
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
string_types = ""
for i in OPENBIS_SESSION.get_object_types().df["code"]:
    string_types += i + ", "

print(string_types)

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data='window.location.replace("home.ipynb")'))


# Function to create sample object inside openBIS using information selected in the app
def create_sample_action(b):
    samples_names = [
        sample.props["name"] for sample in OPENBIS_SESSION.get_objects(type="SAMPLE")
    ]
    if sample_out_name_textbox.value in samples_names:
        display(utils.Javascript(data=f"alert('{'Sample name already exists!'}')"))
        return
    else:
        sample_parents = (
            []
            if material_selector.dropdown.value == -1
            else [material_selector.dropdown.value]
        )
        sample_props = {"name": sample_out_name_textbox.value, "exists": True}
        utils.create_openbis_object(
            OPENBIS_SESSION,
            type="SAMPLE",
            collection="/MATERIALS/SAMPLES/SAMPLE_COLLECTION",
            props=sample_props,
            parents=sample_parents,
        )
        display(utils.Javascript(data="alert('Upload successful!')"))

        # Reset widgets
        material_selection_radio_button.value = "No material"
        sample_out_name_textbox.value = ""


def select_material_radio_change(change):
    material_title = material_selection_radio_button.value
    if material_title == "No material":
        with material_selector:
            clear_output()
            material_selector.dropdown.value = -1
            return

    material_selector.details_textbox.value = ""
    for object_type in CONFIG["slabs"]:
        object_schema = DATA_MODEL["classes"][object_type]
        object_title = object_schema["title"]
        if material_title == object_title:
            material_type = object_type

    with material_selector:
        clear_output()
        display_list = [
            material_selector.dropdown_boxes,
            ipw.HBox([material_selector.details_textbox, material_selector.image_box]),
        ]
        material_selector.load_dropdown_box(material_type)
        display(ipw.VBox(display_list))


# Function to handle changes in the materials dropdown and sample name box
def load_material_metadata_and_sample_name(change):
    material_selector.load_metadata(change)
    # Change sample name box
    material_name = [
        label
        for label, value in material_selector.dropdown.options
        if value == material_selector.dropdown.value
    ][0]
    sample_out_name_textbox.value = (
        utils.convert_datetime_to_string(utils.get_current_datetime())
        + f"_{material_name}"
    )

# Create a sample object in openBIS

## Select material

In [ ]:
material_selection_radio_button.observe(select_material_radio_change, names="value")
material_selector.dropdown.observe(
    load_material_metadata_and_sample_name, names="value"
)
display(material_selection_radio_button)
display(material_selector)

## Sample out

In [ ]:
create_button.on_click(create_sample_action)
quit_button.on_click(close_notebook)
display(sample_out_name_textbox, save_close_buttons_hbox)
display(increase_buttons_size)